# NCAA March Madness Tournament Prediction

In this noteboook, I analysed the important basketball features to predict game outcomes. I experimented with different classical machine learning models such as linear regression, logistic regression, support vector classification and a decision tree with respect to the number of features used and their accuracies to visualise the tournament outcome.

## 1. Import libraries

In [1]:
# Import libraries needed
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import os
import warnings
warnings.filterwarnings('ignore')

## 2. Read and construct the data

## stats.NBA.com emphasizes Four Factors as predictors for winning a NBA game   <br> [Why are the Four factors important?](https://stats.nba.com/help/faq/)
(introduced by Oliver Dean)
- __eFG%:__ effective field goal percentage
- __TOVR:__ turnover ratio
- __OR%:__ offensive rebound percentage
- __FTAR:__ free throw attempt rate

I have included 4 more factors:
<font color='red'>+</font> __DR%:__ defensive rebound percentage
<font color='red'>+</font> __TDR%:__ total defensive rebound percentage
<font color='red'>+</font> __TOR%:__ total offensive rebound percentage
<font color='red'>+</font> __PIE:__ player impact estimate

“High PIE% is highly correlated to winning. In fact, a team’s PIE rating and a team’s winning percentage correlate at an R square of .908 which indicates a "strong" correlation.” – [stats.nba.com](https://stats.nba.com/)

### Calculating the PIE
What is PIE?
It is a simple metric that gives an excellent indication of performance at both the team and player level. It’s a major improvement to our EFF Rating. Notably 2 things changed: (1) We included Personal Fouls, (2) We added a denominator. We feel the key here is the denominator because it acts as an "automatic equalizer". Using the denominator, we find there is no need to consider the "PACE" of the statistics that are being analyzed. In its simplest terms, PIE shows what % of game events did that player or team achieve. The stats being analyzed are your traditional basketball statistics (PTS, REB, AST, TOV, etc..) A team that achieves more than 50% is likely to be a winning team. A player that achieves more than 10% is likely to be better than the average player. A high PIE % is highly correlated to winning. In fact, a team’s PIE rating and a team’s winning percentage correlate at an R square of .908 which indicates a "strong" correlation. We’ve introduced this statistic because we feel it incorporates a bit of defense into the equation. When a team misses a shot, all 5 players on the other team’s PIE rating goes up. The formula for PIE can be found here in the Glossary.
Source: [What is PIE?](https://stats.nba.com/help/faq/)
<br>
Formula: (PTS + FGM + FTM - FGA - FTA + DREB + (.5 * OREB) + AST + STL + (.5 * BLK) - PF - TO) / (GmPTS + GmFGM + GmFTM - GmFGA - GmFTA + GmDREB + (.5 * GmOREB) + GmAST + GmSTL + (.5 * GmBLK) - GmPF - GmTO)

In [2]:
stage1dir = Path('mens-machine-learning-competition-2020/MDataFiles_Stage2')

In [3]:
def load_and_construct_features_in_df(year):
    """
    Input: year: Season year, e.g. 2017
    Output: df: DataFrame with constructed features that can be used for training a ML model.
    """
    
    #Read-in teams' tournament seedings by year
    df_teams_seeds = pd.read_csv(stage1dir/'MNCAATourneySeeds.csv')
    # Strip letters in seed column an return an integer
    df_teams_seeds['Seed'] = df_teams_seeds['Seed'].apply(lambda x: int(x[1:3]))
    
    #Read-in the regular season detailed results dataset
    df_tournament_results = pd.read_csv(stage1dir/'MRegularSeasonDetailedResults.csv')

    #Add 'WSeed','LSeed' feature to Detailed Tournament list by merging with renamed lists
    # and calculate SeedDifference and add as a feature to df_final
    df_renamed_win = df_teams_seeds.rename(columns={'TeamID':'WTeamID','Seed':'WSeed'})
    df_renamed_loss = df_teams_seeds.rename(columns={'TeamID':'LTeamID','Seed':'LSeed'})
    df_mixed = df_tournament_results.merge(df_renamed_win, how='left', on=['Season','WTeamID'])
    df_final = df_mixed.merge(df_renamed_loss, how='left', on=['Season','LTeamID'])
    
    ### FEATURE CONSTRUCTION STARTS HERE ###
    df_final['SeedDifference'] = df_final['WSeed'] - df_final['LSeed'] 
    df_final['ScoreDifference'] = df_final['WScore'] - df_final['LScore'] 
    #df_final.head()

    # DTOVR (Difference in 'Turnover Ratio'): 100 * TOV / (FGA + (0.44 * FTA) + AST + TOV)
    df_final['WTOVR'] = 100* df_final['WTO']/(df_final['WFGA'] + (0.44*df_final['WFTA']) + df_final['WAst'] + df_final['WTO'])
    df_final['LTOVR'] = 100* df_final['LTO']/(df_final['LFGA'] + (0.44*df_final['LFTA']) + df_final['LAst'] + df_final['LTO'])
    df_final['DTOVR'] = df_final['WTOVR'] - df_final['LTOVR']
    # DeFG% (Difference in 'effective FieldGoal percentage'):  (FGM + 0.5 * 3PM) / FGA.
    df_final['WeFG%'] = (df_final['WFGM'] + 0.5*df_final['WFGM3'])/(df_final['WFGA'] + df_final['WFGA3'])
    df_final['LeFG%'] = (df_final['LFGM'] + 0.5*df_final['LFGM3'])/(df_final['LFGA'] + df_final['LFGA3'])
    df_final['DeFG%'] = df_final['WeFG%'] - df_final['LeFG%']

    #df_final['DeFG%'].describe()
    # DFTAR (Difference in 'FreeThrow Attempt rate'):  FTA / FGA.
    df_final['WFTAR'] = df_final['WFTA']/df_final['WFGA']
    df_final['LFTAR'] = df_final['LFTA']/df_final['LFGA']
    df_final['DFTAR'] = df_final['WFTAR'] - df_final['LFTAR']
    # DOR% (Difference in 'Offensive rebound %'):  WOR / LDR+WOR. Notice that we divide by the 
    # a) number of OffensiveRebounds + number of OpponentsDefensiveRebounds (We look at one side of the board).
    df_final['WOR%'] = df_final['WOR']/(df_final['LDR'] + df_final['WOR'])
    df_final['LOR%'] = df_final['LOR']/(df_final['LOR'] + df_final['WDR'])
    df_final['DOR%'] = df_final['WOR%'] - df_final['LOR%']
    # b) DDR% (Difference in 'Defensive rebound %'):  WDR / LOR+WDR.
    df_final['WDR%'] = df_final['WDR']/(df_final['LOR'] + df_final['WDR'])
    df_final['LDR%'] = df_final['LDR']/(df_final['LDR'] + df_final['WOR'])
    df_final['DDR%'] = df_final['WDR%'] - df_final['LDR%']
    # c) DTDR% (Difference in 'Total Defensive rebound %'):  WDR / LDR+WDR.
    df_final['WTDR%'] = df_final['WDR']/(df_final['LDR'] + df_final['WDR'])
    df_final['LTDR%'] = df_final['LDR']/(df_final['LDR'] + df_final['WDR'])
    df_final['DTDR%'] = df_final['WTDR%'] - df_final['LTDR%']
    # d) DTOR% (Difference in 'Total Offensive rebound %'):  WOR / LOR+WOR.
    df_final['WTOR%'] = df_final['WOR']/(df_final['LOR'] + df_final['WOR'])
    df_final['LTOR%'] = df_final['LOR']/(df_final['LOR'] + df_final['WOR'])
    df_final['DTOR%'] = df_final['WTOR%'] - df_final['LTOR%']
    # Adding PIE value to df_final
    df_final['WPIE'] = (df_final['WScore'] + df_final['WFGM'] + df_final['WFTM'] - df_final['WFGA'] 
                        - df_final['WFTA'] + df_final['WDR'] + (0.5*df_final['WOR']) + df_final['WAst'] 
                        + df_final['WStl'] + (0.5*df_final['WBlk']) - df_final['WPF'] - df_final['WTO']
                        )/(
                        df_final['WScore'] + df_final['LScore'] + df_final['WFGM'] + df_final['LFGM'] 
                        + df_final['WFTM'] + df_final['LFTM'] - df_final['WFGA'] - df_final['LFGA'] 
                        + df_final['WDR'] + df_final['LDR'] + (0.5*df_final['WOR'] + 0.5*df_final['LOR']) 
                        + df_final['WAst'] + df_final['LAst'] + df_final['WStl'] + df_final['LStl'] 
                        + (0.5*df_final['WBlk'] + 0.5*df_final['LBlk']) - df_final['WPF'] - df_final['LPF']
                        - df_final['WTO'] - df_final['WTO']
                        )
    df_final['LPIE'] = (df_final['LScore'] + df_final['LFGM'] + df_final['LFTM'] - df_final['LFGA']
                     - df_final['LFTA'] + df_final['LDR'] + (0.5*df_final['LOR']) + df_final['LAst']
                     + df_final['LStl'] + (0.5*df_final['LBlk']) - df_final['LPF'] - df_final['LTO']
                     )/(
                     df_final['WScore'] + df_final['LScore'] + df_final['WFGM'] + df_final['LFGM'] 
                     + df_final['WFTM'] + df_final['LFTM'] - df_final['WFGA'] - df_final['LFGA'] 
                     + df_final['WDR'] + df_final['LDR'] + (0.5*df_final['WOR'] + 0.5*df_final['LOR']) 
                     + df_final['WAst'] + df_final['LAst'] + df_final['WStl'] + df_final['LStl'] 
                     + (0.5*df_final['WBlk'] + 0.5*df_final['LBlk'])  - df_final['WPF'] - df_final['LPF']
                     - df_final['WTO'] - df_final['WTO']
                     )
    df_final['DPIE'] = df_final['WPIE'] - df_final['LPIE']
    
    #only getting the games from the year inputted
    df_final = df_final[df_final['Season'] == year]
    df_final = df_final.reset_index(drop=True)
    
    return df_final

In [4]:
year1 = 2019
dcols=['DTOVR', 'DeFG%', 'DFTAR', 'DDR%', 'DOR%', 'DTDR%', 'DTOR%', 'DPIE']
cols=['TOVR', 'eFG%', 'FTAR', 'DR%', 'OR%', 'TDR%', 'TOR%', 'PIE']

In [5]:
df_final = load_and_construct_features_in_df(year1)
df_final.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,DDR%,WTDR%,LTDR%,DTDR%,WTOR%,LTOR%,DTOR%,WPIE,LPIE,DPIE
0,2019,1,1104,82,1380,62,H,0,27,55,...,0.169444,0.591837,0.408163,0.183673,0.592593,0.407407,0.185185,0.478431,0.105882,0.372549
1,2019,1,1113,102,1168,94,H,0,33,78,...,0.208081,0.520548,0.479452,0.041096,0.740741,0.259259,0.481481,0.333333,0.271186,0.062147
2,2019,1,1119,73,1265,69,H,0,22,51,...,-0.108504,0.448980,0.551020,-0.102041,0.400000,0.600000,-0.200000,0.341060,0.268212,0.072848
3,2019,1,1120,101,1375,58,H,0,38,74,...,0.241935,0.547619,0.452381,0.095238,0.703704,0.296296,0.407407,0.577160,0.145062,0.432099
4,2019,1,1123,86,1232,69,H,0,32,65,...,0.035354,0.565217,0.434783,0.130435,0.470588,0.529412,-0.058824,0.500000,0.245223,0.254777


## Extracting training data

In [6]:
# Extracting training data from the final dataframe
def get_training_data(df_final, columns):
    """
    Input:  df_final: Dataframe with all features that can be used for estimating game results.
            Columns: List of column labels. Column labels must be present in df_final.
    Output: X_train, Y_train datasets, which can be directly fed into the LogReg model
    """
    for feature in columns:
        if feature not in df_final.columns:
            raise Exception('The feature {}'.format(feature)+' is not in the DataFrame. All features must be labels of a column of the dataframe.')
    
    df_wins = pd.DataFrame()
    df_losses = pd.DataFrame()
    for feature in columns:
        df_wins[str(feature)] = df_final[str(feature)]
        df_losses[str(feature)] = -df_final[str(feature)]
    df_wins['Result'] = 1
    df_losses['Result'] = 0

    training_data = pd.concat([df_losses, df_wins], axis=0)

    if len(columns) == 1:
        X_train = training_data[columns].values.reshape(-1,1)
    else:
        X_train = training_data[columns].values
    Y_train = training_data.Result.values
    return (X_train, Y_train)

In [7]:
X_train, Y_train = get_training_data(df_final, dcols)
X_train.shape, Y_train.shape

((10926, 8), (10926,))

## Get training data and setup ML algorithm
What will be the input for my classifier? The PIE value shows the strongest correlation, hence associating the difference of PIE value with 0 or 1 could be one way or the values themself.

In [8]:
def train_ML_model(X_train, Y_train, cv, model):
    
    '''
    Input:  X_train, Y_train: X and Y training data
            cv: Cross-validation integer value
            model: A string of either: 'LinReg','LogReg','DecTree','SVC'.
    Output: GridSearchedObject, ClassifierObject, LogLoss, Regularization Parameter C
    '''
    
    if model not in ['LinReg','LogReg','DecTree','SVC']:
        raise Exception('Model needs to be LinReg, LogReg, DecTree or SVC')

    X_train, Y_train = shuffle(X_train, Y_train)
    
    #Linear Regression
    if model == 'LinReg':
        linreg = LinearRegression()
        params = {
            'fit_intercept':[True,False], 
            'normalize':[True,False], 
            'copy_X':[True, False],
        }
        clf = GridSearchCV(linreg, params,cv=cv)
        clf.fit(X_train, Y_train)
        print('Best score: {:.4}'.format(clf.best_score_ ))
        C_dummy = 0
        return clf, linreg, clf.best_score_ , C_dummy
    
    #Logistic Regression
    if model == 'LogReg':
        logreg = LogisticRegression()
        C = [int(x) for x in np.linspace(start=-15, stop=15, num=16)]
        multi_class = ['auto', 'multinomial']
        solver = ['newton-cg', 'sag', 'saga', 'lbfgs', 'liblinear']
        class_weight = ['balanced', None]
        penalty = ['l2']
        params = {'C': C,
                  'multi_class': multi_class, 
                  'solver': solver, 
                  'class_weight': class_weight, 
                  'penalty': penalty,
        }
        clf = GridSearchCV(logreg, params, scoring='accuracy', refit=True, cv=cv)
        clf.fit(X_train, Y_train)
        print('Best accuracy: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))
        return clf, logreg, clf.best_score_, clf.best_params_['C']
    
    #SVC
    if model == 'SVC':
        SVCreg = SVC(probability=True)
        params = {'C': np.logspace(start=-2, stop=3, num=10), 
                  'kernel':['rbf'],
        }
        clf = GridSearchCV(SVCreg, params, scoring='accuracy', refit=True, cv=cv)
        clf.fit(X_train, Y_train)
        print('Best accuracy: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))
        return clf, SVCreg, clf.best_score_, clf.best_params_['C']
    
    #Decision Tree
    if model == 'DecTree':
        Tree = DecisionTreeClassifier()
        params = {'max_depth': range(1,8),
                  'min_samples_split': np.arange(2,7), 
                  'min_samples_leaf':np.arange(1,5), 
                  'max_features': np.arange(1,X_train.shape[1]+1)
        }
        clf = GridSearchCV(Tree, params, scoring='accuracy', refit=True, cv=cv)
        clf.fit(X_train, Y_train)
        print('Best accuracy: {:.4}, with best max_depth: {}, min_samples_split: {}, min_samples_leaf: {}, max_features_split: {}'\
              .format(clf.best_score_, clf.best_params_['max_depth'], clf.best_params_['min_samples_split'], clf.best_params_['min_samples_leaf'], clf.best_params_['max_features']))
        return clf, Tree, clf.best_score_, clf.best_params_['max_depth']

In [9]:
clf, reg, score, C = train_ML_model(X_train, Y_train, cv=3, model='LogReg')

Best accuracy: 0.9524, with best C: 9


## Make a prediction
#### So now in order to make a prediction I have to take in e.g. the Difference of the Player Impact Estimate (DPIE) of the two teams playing:
There are different options to do this:
- Take the average over every season game (pretournament)
- Take the average over the past 8 games
- Just include previous games of these two team

#### For now, we will take the average value over every regular season game

In [10]:
# Load the list that made it to the tournament
def generate_tournament_team_list(year):
    """
    List of teams will be generated that made it into the Tournament.
    Output: List and copy of list.
    """
    df_teams = pd.read_csv(stage1dir/'MNCAATourneyCompactResults.csv').query('Season=='+str(year))
    df_teams = df_teams.sort_values(by=['WTeamID','LTeamID'])
    #print(df_teams)

    a = df_teams.drop_duplicates(subset=['WTeamID'])['WTeamID'] # List of winning teams
    b = df_teams.drop_duplicates(subset=['LTeamID'])['LTeamID'] # List of losing Teams
    Team_IDs = a.append(b,ignore_index=True).drop_duplicates().values.tolist() # Add both lists and drop duplicates
    Team_IDs_copy = a.append(b,ignore_index=True).drop_duplicates().values.tolist() # List of TeamIDs
    Team_IDs_copy = sorted(Team_IDs_copy)
    Team_IDs = sorted(Team_IDs)
#     print ('Number of teams playing in '+str(year)+' Tournament: '+ str(len(Team_IDs))+'/68 (64 + The first four).')
    return Team_IDs, Team_IDs_copy

In [11]:
Team_IDs, Team_IDs_copy = generate_tournament_team_list(year1)

In [12]:
def generate_df_for_prediction(df_final, columns, year):
    '''
    Input:  df_dinal: DF that contains features which will be used as estimators. Require 'WTeamID' and 'LTeamID' features.
            columns: List of features that will be averaged. Columns must be contained in df_final
    Output: df with TeamID and averaged feature over the past season.
    '''
    for feature in columns:
        if 'W'+feature not in df_final.columns or 'L'+feature not in df_final.columns:
            raise Exception('The feature {}'.format(feature)+' is not in the DataFrame. All features must be labels of a column of the dataframe.')

    Team_IDs, Team_IDs_copy = generate_tournament_team_list(year)
            
    df_for_prediction = pd.DataFrame(columns=['ID'] + columns)
    counter = 0
    for ID in Team_IDs:
        df_for_prediction.at[counter, 'ID'] = str(ID)
        for feature in columns:
            a = df_final[df_final['WTeamID']==ID]['W'+str(feature)] # Gives a colum of a specific team with the feature called 'feature'.
            b = df_final[df_final['LTeamID']==ID]['L'+str(feature)]
            AVERAGE = a.append(b,ignore_index=True).mean()
            df_for_prediction.at[counter, feature] = AVERAGE
            #df_for_prediction = df_for_prediction.append({'ID':str(ID), str(feature):AVERAGE}, ignore_index=True)
            del a
            del b
            del AVERAGE
        counter += 1
    return df_for_prediction

In [13]:
df_for_prediction = generate_df_for_prediction(df_final, cols, year1)
df_for_prediction.head()
#df_for_prediction

,ID,TOVR,eFG%,FTAR,DR%,OR%,TDR%,TOR%,PIE
0,1101,12.8607,0.392524,0.348824,0.725586,0.274224,0.489166,0.513218,0.401968
1,1113,13.7916,0.377476,0.438758,0.734541,0.324304,0.524894,0.550185,0.375604
2,1120,12.762,0.362077,0.320622,0.67478,0.319014,0.474348,0.515858,0.409937
3,1124,14.3156,0.364489,0.334952,0.714692,0.378682,0.531909,0.573793,0.391544
4,1125,11.2741,0.406128,0.290242,0.759502,0.254261,0.535983,0.485218,0.482154


In [14]:
# Make a DF which provides Game_ID and the difference of an indicator/feature, e.g. 'DeFG%'
def generate_prediction_features(df_for_prediction, columns, year):
    """
    Input:  df_for_prediction: Obtained from generate_df_for_prediction(df_final, columns).
            columns: Features containes in df_for_prediction e.g. ['eFG%','PIE']
            year: Seaon year
    Outout: X_Prediction columns
    """
    for feature in columns:
        if feature not in df_for_prediction.columns:
            raise Exception('The feature {}'.format(feature)+' is not in the DataFrame. All features must be labels of a column of the dataframe.')
    # ['eFG%','PIE']
    col = ['ID','Season','Team1','Team2']
    for feature in columns:
        cols = col + [feature+'1'] + [feature+'2'] + ['D'+feature]
    #cols = ['ID','Season','Team1','Team2','eFG%1','eFG%2','DeFG%']
    df_prediction = pd.DataFrame(columns=cols)
    #Tournament list need to be regenerate every time generater predictions is called because the teams are removed from the list
    Team_IDs, Team_IDs_copy = generate_tournament_team_list(year=year)
#     print('Number of teams playing in '+str(year)+' Tournnament: '+ str(len(Team_IDs))+'/68 (64 + The first four).')
    dummy_a = Team_IDs
    dummy_b = Team_IDs_copy
    counter = 0
    for ID1 in dummy_a:
        #print ID
        dummy_b.remove(ID1)
        for ID2 in dummy_b:
            Game_ID = str(year)+'_'+str(ID1)+'_'+str(ID2)
            df_prediction.at[counter, 'ID'] = Game_ID # Add Game_ID value to row = counter and cell='ID' 
            df_prediction.at[counter, 'Season'] = year
            df_prediction.at[counter, 'Team1'] = ID1
            df_prediction.at[counter, 'Team2'] = ID2
            for feature in columns:
                dummy_var_a = df_for_prediction[df_for_prediction['ID'] == str(ID1)][str(feature)].values[0]
                dummy_var_b = df_for_prediction[df_for_prediction['ID'] == str(ID2)][str(feature)].values[0]
                df_prediction.at[counter, feature+'1'] = dummy_var_a
                df_prediction.at[counter, feature+'2'] = dummy_var_b
                df_prediction.at[counter, 'D'+feature] = dummy_var_a - dummy_var_b
            counter += 1
    for features in col: # convert values in features column to float values
        df_prediction['D'+feature] = df_prediction['D'+feature].apply(lambda x: float(x))
    #print len(df_prediction)
    return df_prediction

In [15]:
df_pred = generate_prediction_features(df_for_prediction, cols, year1)
df_pred
#df_pred['DeFG%'].describe()

,ID,Season,Team1,Team2,PIE1,PIE2,DPIE,TOVR1,TOVR2,DTOVR,...,DDR%,OR%1,OR%2,DOR%,TDR%1,TDR%2,DTDR%,TOR%1,TOR%2,DTOR%
0,2019_1101_1113,2019,1101,1113,0.401968,0.375604,0.026365,12.860742,13.791623,-0.930881,...,-0.008955,0.274224,0.324304,-0.050080,0.489166,0.524894,-0.035727,0.513218,0.550185,-0.036968
1,2019_1101_1120,2019,1101,1120,0.401968,0.409937,-0.007968,12.860742,12.762008,0.098734,...,0.050806,0.274224,0.319014,-0.044790,0.489166,0.474348,0.014819,0.513218,0.515858,-0.002641
2,2019_1101_1124,2019,1101,1124,0.401968,0.391544,0.010424,12.860742,14.315633,-1.454891,...,0.010894,0.274224,0.378682,-0.104458,0.489166,0.531909,-0.042742,0.513218,0.573793,-0.060576
3,2019_1101_1125,2019,1101,1125,0.401968,0.482154,-0.080185,12.860742,11.274073,1.586669,...,-0.033916,0.274224,0.254261,0.019963,0.489166,0.535983,-0.046816,0.513218,0.485218,0.027999
4,2019_1101_1133,2019,1101,1133,0.401968,0.36345,0.038519,12.860742,14.483708,-1.622966,...,-0.007369,0.274224,0.268269,0.005955,0.489166,0.506850,-0.017684,0.513218,0.497433,0.015784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2019_1449_1459,2019,1449,1459,0.390017,0.450061,-0.060044,15.315940,12.114010,3.201930,...,-0.100897,0.288639,0.322776,-0.034137,0.494001,0.531015,-0.037014,0.442336,0.571679,-0.129343
2274,2019_1449_1463,2019,1449,1463,0.390017,0.454026,-0.064009,15.315940,13.474279,1.841661,...,-0.102646,0.288639,0.258883,0.029756,0.494001,0.550910,-0.056909,0.442336,0.468376,-0.026040
2275,2019_1458_1459,2019,1458,1459,0.443219,0.450061,-0.006843,11.062754,12.114010,-1.051256,...,-0.019617,0.241386,0.322776,-0.081391,0.514333,0.531015,-0.016683,0.452339,0.571679,-0.119340
2276,2019_1458_1463,2019,1458,1463,0.443219,0.454026,-0.010807,11.062754,13.474279,-2.411525,...,-0.021366,0.241386,0.258883,-0.017498,0.514333,0.550910,-0.036577,0.452339,0.468376,-0.016037


In [16]:
def get_predcitions(df_pred, columns, classifier):
    '''Input:   df_pred: DF which containes predictors, e.g. 'DeFG%','DPIE' 
                columns: Colums with labels of predictors, e.g. ['DeFG%','DPIE']
                classifier: A classifier from the machine learning section
       Output: Dataframe with updated predictions based of ML classifier model.
    '''
    for feature in columns:
        if feature not in df_pred.columns:
            raise Exception('The feature {}'.format(feature)+' is not in the DataFrame. All features must be labels of a column of the dataframe.')
    if len(columns) == 1:
        X_pred_columns = df_pred[columns].values.reshape(-1,1)
    elif len(columns) > 1:
        X_pred_columns = df_pred[columns].values
    elif len(columns) == 0:
        return 'Columns should be a list of feature and at least contain 1 feature.'
    
    if type(reg) == type(LinearRegression()):
        X_pred = classifier.predict(X_pred_columns)
    else:
        X_pred = classifier.predict_proba(X_pred_columns)[:,1]
    #print X_pred
    df_pred['Pred'] = X_pred
    df_pred['PredWL'] = df_pred['Pred'].apply(lambda x: 0 if x <= 0.5 else 1)
    return df_pred

In [17]:
df_predicted_values = get_predcitions(df_pred, dcols, classifier=clf)
df_predicted_values

,ID,Season,Team1,Team2,PIE1,PIE2,DPIE,TOVR1,TOVR2,DTOVR,...,OR%2,DOR%,TDR%1,TDR%2,DTDR%,TOR%1,TOR%2,DTOR%,Pred,PredWL
0,2019_1101_1113,2019,1101,1113,0.401968,0.375604,0.026365,12.860742,13.791623,-0.930881,...,0.324304,-0.050080,0.489166,0.524894,-0.035727,0.513218,0.550185,-0.036968,0.784407,1
1,2019_1101_1120,2019,1101,1120,0.401968,0.409937,-0.007968,12.860742,12.762008,0.098734,...,0.319014,-0.044790,0.489166,0.474348,0.014819,0.513218,0.515858,-0.002641,0.517783,1
2,2019_1101_1124,2019,1101,1124,0.401968,0.391544,0.010424,12.860742,14.315633,-1.454891,...,0.378682,-0.104458,0.489166,0.531909,-0.042742,0.513218,0.573793,-0.060576,0.732626,1
3,2019_1101_1125,2019,1101,1125,0.401968,0.482154,-0.080185,12.860742,11.274073,1.586669,...,0.254261,0.019963,0.489166,0.535983,-0.046816,0.513218,0.485218,0.027999,0.073202,0
4,2019_1101_1133,2019,1101,1133,0.401968,0.36345,0.038519,12.860742,14.483708,-1.622966,...,0.268269,0.005955,0.489166,0.506850,-0.017684,0.513218,0.497433,0.015784,0.883526,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2019_1449_1459,2019,1449,1459,0.390017,0.450061,-0.060044,15.315940,12.114010,3.201930,...,0.322776,-0.034137,0.494001,0.531015,-0.037014,0.442336,0.571679,-0.129343,0.056170,0
2274,2019_1449_1463,2019,1449,1463,0.390017,0.454026,-0.064009,15.315940,13.474279,1.841661,...,0.258883,0.029756,0.494001,0.550910,-0.056909,0.442336,0.468376,-0.026040,0.068878,0
2275,2019_1458_1459,2019,1458,1459,0.443219,0.450061,-0.006843,11.062754,12.114010,-1.051256,...,0.322776,-0.081391,0.514333,0.531015,-0.016683,0.452339,0.571679,-0.119340,0.322173,0
2276,2019_1458_1463,2019,1458,1463,0.443219,0.454026,-0.010807,11.062754,13.474279,-2.411525,...,0.258883,-0.017498,0.514333,0.550910,-0.036577,0.452339,0.468376,-0.016037,0.371383,0


## Validation: Check how many predictions are correct

In [18]:
def check_correct_answers(df_predicted_values ,columns, year, print_df_head = False):
    '''
    Input: df_predicted_values: DataFrame with ID (year_team1_team2) and predictions (0 to 1) or predictions as win or loss (0 or 1)
           columns: List of features, e.g. ['eFG%','PIE'].
           year: Season year
    Output: Printed number of games predicted correctly.
    '''
    directory = os.getcwd().replace('\\','/')
    # Load in results
    df_results = pd.read_csv(stage1dir/'MNCAATourneyCompactResults.csv').query('Season=='+str(year))
    df_results['Resu'] = 1
#     df_results.head()
    df_results = df_results.sort_values(by=['WTeamID','LTeamID'])
    df_results['ID'] = df_results['Season'].apply(lambda x: str(x))+'_'+df_results['WTeamID'].apply(lambda x: str(x))+'_'+df_results['LTeamID'].apply(lambda x: str(x))
   
    #iterate over df_results['ID'] and swap if team1<team2 (Game_ID = Year_Team1_Team2)
    for index, row in df_results.iterrows():
        team1, team2 = row['ID'].split('_')[1], row['ID'].split('_')[2]
        if float(team1) > float(team2): #This is the case that gives NAN right now
            #print row['ID'].split('_')
            swapped_ID = row['ID'].split('_')[0]+'_'+team2+'_'+team1
            df_results.at[index,'ID'] = swapped_ID
            df_results.at[index,'Resu'] = 0
    #df_results.head()

    # Merge df_results with df_predicted_values
    df_pred_result = df_results.merge(df_predicted_values, how='left', on=['ID'])
    cols = ['Season_x','ID','WTeamID','Team1','WScore','LTeamID','LScore','Team2','Resu','PredWL','Pred']
    for feature in columns:
        cols = cols + [feature+'1', feature+'2', 'D'+feature] #'eFG%1','eFG%2','DeFG%','PIE1', 'PIE2', 'DPIE'
    df_pred_result = df_pred_result[cols]
    if print_df_head == True:
        print(df_pred_result.head())

    # Number of correctly estimated games y
    numOfWins = (df_pred_result['PredWL'] == df_pred_result['Resu']).sum()
    gamesPlayed = float(len((df_pred_result['PredWL'])))
    numOfWinsPerc = round(100*(numOfWins/gamesPlayed),1)
    print(str(year)+' Tournament games predicted correctly: '+ str(numOfWins)+' out of '+ str(gamesPlayed)+'. ('+str(numOfWinsPerc)+'%)')
    return numOfWins, gamesPlayed, numOfWinsPerc

In [19]:
numOfWins, gamesPlayed, numOfWinsPerc = check_correct_answers(df_predicted_values, cols, year1, print_df_head = False)
#print numOfWins, gamesPlayed, numOfWinsPerc

2019 Tournament games predicted correctly: 44 out of 67.0. (65.7%)


# Wrapping all functions until now

In [20]:
# Summarize all functions
df_final = load_and_construct_features_in_df(year1)
#print df_final.columns

In [21]:
def main(df_final, columns, year, model):
    """Input: Columns: List in the form of ['eFG%','PIE'] instead of ['DeFG%','DPIE']
              model: model: A string of either: 'LinReg','LogReg','DecTree','SVC'.
    """
    columns1 = columns # e.g., ['eFG%','PIE']
    columns2 = list(map(lambda x: 'D'+x, columns)) # e.g., ['DeFG%','DPIE']
    
    X_train, Y_train = get_training_data(df_final, columns = columns2)
    clf, reg, score, C = train_ML_model(X_train, Y_train, cv=3, model=model)

    df_for_prediction = generate_df_for_prediction(df_final, columns= columns1, year=year)
    df_pred = generate_prediction_features(df_for_prediction, columns= columns1, year=year)
    df_predicted_values = get_predcitions(df_pred, columns= columns2, classifier=clf)

    numOfWins, gamesPlayed, numOfWinsPerc = check_correct_answers(df_predicted_values, columns = columns1, year = year, print_df_head = False)
    return year, columns, numOfWins, gamesPlayed, numOfWinsPerc, score, C, clf
# Note: Before 2011 only 65 Teams played in the NBA.

In [22]:
year, columns, numOfWins, gamesPlayed, numOfWinsPerc, score, C, clf = main(df_final, cols, year1, model='LogReg')

Best accuracy: 0.9514, with best C: 3
2019 Tournament games predicted correctly: 44 out of 67.0. (65.7%)


##  Train and evaluate different models for different features (fixed year)

In [27]:
# This cell was run offline
#Get model parameters for one year but different features
import itertools
df_stats = pd.DataFrame(columns=['Season','Columns','WinsPred','GamesPlayed','WinsPred%','Score','C','Coef weight'])
counter = 0

In [28]:
for year in range(2019, 2020): # for one year
    df_final = load_and_construct_features_in_df(year)
    model = 'LogReg' # model: A string of either: 'LinReg','LogReg','DecTree','SVC'
    columns_main = cols
    # The combination of column elements is done here, with 7,6,5,4 elements in the column vector.
    for k in [8,7,]: #[6,5,4,3,2,1]
        list_of_feature_combinations = list(itertools.combinations(columns_main,k))
        #print len(list_of_feature_combinations)
        #print list_of_feature_combinations
        for element in list_of_feature_combinations:
            col = sorted(list(element))
            print(col)
            year, columns, numOfWins, gamesPlayed, numOfWinsPerc, score, C, clf= main(df_final, columns=col, year=year, model=model)
            df_stats.at[counter,'Season'] = year
            df_stats.at[counter,'Columns'] = columns
            df_stats.at[counter,'WinsPred'] = numOfWins
            df_stats.at[counter,'GamesPlayed'] = gamesPlayed
            df_stats.at[counter,'WinsPred%'] = numOfWinsPerc
            df_stats.at[counter,'Score'] = score
            df_stats.at[counter,'C'] = C
            if model == 'SVC':
                df_stats.at[counter,'Coef weight'] = 0
            elif model == 'DecTree':
                df_stats.at[counter,'Coef weight'] = clf.best_estimator_.feature_importances_ 
            elif model == 'LogReg':
                df_stats.at[counter,'Coef weight'] = clf.best_estimator_.coef_[0]
            elif model == 'LinReg':
                df_stats.at[counter,'Coef weight'] = clf.best_estimator_.coef_
            counter += 1
df_stats

['DR%', 'FTAR', 'OR%', 'PIE', 'TDR%', 'TOR%', 'TOVR', 'eFG%']
Best accuracy: 0.9521, with best C: 7
2019 Tournament games predicted correctly: 44 out of 67.0. (65.7%)
['DR%', 'FTAR', 'OR%', 'TDR%', 'TOR%', 'TOVR', 'eFG%']
Best accuracy: 0.9232, with best C: 13
2019 Tournament games predicted correctly: 45 out of 67.0. (67.2%)
['DR%', 'FTAR', 'OR%', 'PIE', 'TDR%', 'TOVR', 'eFG%']
Best accuracy: 0.9521, with best C: 3
2019 Tournament games predicted correctly: 44 out of 67.0. (65.7%)
['DR%', 'FTAR', 'OR%', 'PIE', 'TOR%', 'TOVR', 'eFG%']
Best accuracy: 0.9517, with best C: 9
2019 Tournament games predicted correctly: 44 out of 67.0. (65.7%)
['DR%', 'FTAR', 'PIE', 'TDR%', 'TOR%', 'TOVR', 'eFG%']
Best accuracy: 0.9521, with best C: 15
2019 Tournament games predicted correctly: 43 out of 67.0. (64.2%)
['FTAR', 'OR%', 'PIE', 'TDR%', 'TOR%', 'TOVR', 'eFG%']
Best accuracy: 0.9522, with best C: 7
2019 Tournament games predicted correctly: 45 out of 67.0. (67.2%)
['DR%', 'OR%', 'PIE', 'TDR%', 'TO

,Season,Columns,WinsPred,GamesPlayed,WinsPred%,Score,C,Coef weight
0,2019,"[DR%, FTAR, OR%, PIE, TDR%, TOR%, TOVR, eFG%]",44,67,65.7,0.952133,7,"[1.5565887624585422, 0.517093170409054, 1.5565..."
1,2019,"[DR%, FTAR, OR%, TDR%, TOR%, TOVR, eFG%]",45,67,67.2,0.923211,13,"[-0.12447812972630974, 4.837359272854062, -0.1..."
2,2019,"[DR%, FTAR, OR%, PIE, TDR%, TOVR, eFG%]",44,67,65.7,0.952133,3,"[1.8427673007884366, 0.5592117677466204, 1.842..."
3,2019,"[DR%, FTAR, OR%, PIE, TOR%, TOVR, eFG%]",44,67,65.7,0.951675,9,"[-1.5289020819172616, 1.0145943258063892, -1.5..."
4,2019,"[DR%, FTAR, PIE, TDR%, TOR%, TOVR, eFG%]",43,67,64.2,0.952133,15,"[5.454879049851527, 1.0236188247786577, 34.440..."
5,2019,"[FTAR, OR%, PIE, TDR%, TOR%, TOVR, eFG%]",45,67,67.2,0.952224,7,"[0.5142466535742232, 2.601583849659469, 17.095..."
6,2019,"[DR%, OR%, PIE, TDR%, TOR%, TOVR, eFG%]",43,67,64.2,0.951583,3,"[0.99284898281693, 0.9928489828169242, 27.8008..."
7,2019,"[DR%, FTAR, OR%, PIE, TDR%, TOR%, TOVR]",43,67,64.2,0.948746,11,"[0.7454447090333691, 0.10473996776616154, 0.74..."
8,2019,"[DR%, FTAR, OR%, PIE, TDR%, TOR%, eFG%]",44,67,65.7,0.950577,3,"[0.8891964732306269, 1.0288311715392302, 0.889..."


In [ ]:
#df_stats.to_csv('ModelFeatureAnalysis_of_LinReg_1-6feat_2016.csv', index=False)
df_stats.to_csv('ModelFeatureAnalysis_of_LogReg_1-6feat_2016.csv', index=False)
#df_stats.to_csv('ModelFeatureAnalysis_of_SVC_1-6feat_2016.csv', index=False)
#df_stats.to_csv('ModelFeatureAnalysis_of_Tree_1-6feat_2016.csv', index=False)